In [2]:
class SimpleClass():
    
    def __init__(self, name):
        print("hello " + name)
        
    def yell(self):
        print("YELLING")

In [3]:
s = "world"

In [4]:
type(s)

str

In [5]:
x = SimpleClass("Igor")

hello Igor


In [6]:
x

In [7]:
x.yell()

YELLING


In [8]:
class ExtendedClass(SimpleClass):
    
    def __init__(self):
        super().__init__("Igor")
        print("EXTEND")

In [9]:
y = ExtendedClass()

hello Igor
EXTEND


In [10]:
y.yell()

YELLING


## Operation

In [11]:
class Operation():
    def __init__(self, input_nodes=[]):
        self.input_nodes = input_nodes
        self.output_nodes = []
        
        for node in input_nodes:
            node.output_nodes.append(self)
            
        _default_graph.operations.append(self)
    
    def compute(self):
        pass

In [12]:
class add(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var + y_var

In [13]:
class multiply(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var * y_var

In [14]:
class matmul(Operation):
    def __init__(self, x, y):
        super().__init__([x, y])
        
    def compute(self, x_var, y_var):
        self.inputs = [x_var, y_var]
        return x_var.dot(y_var)

In [15]:
class Placeholder():
    
    def __init__(self):
        
        self.output_nodes = []
        _default_graph.placeholders.append(self)
        

In [16]:
class Variable():
    
    def __init__(self, initial_value=None):
        
        self.value = initial_value
        self.output_nodes = []
        
        _default_graph.variables.append(self)
    

In [17]:
class Graph():
    
    def __init__(self):
        
        self.operations = []
        self.placeholders = []
        self.variables = []
        
    def set_as_default(self):
        
        global _default_graph
        _default_graph = self

z = Ax + b

A = 10

b = 1

z = 10x + 1

In [18]:
g = Graph()

In [19]:
g.set_as_default()

In [20]:
A = Variable(10)

In [21]:
b = Variable(1)

In [22]:
x = Placeholder()

In [23]:
y = multiply(A, x)

In [24]:
z = add(y, b)

In [34]:
def traverse_postorder(operation):
    """ 
    PostOrder Traversal of Nodes. Basically makes sure computations are done in 
    the correct order (Ax first , then Ax + b). Feel free to copy and paste this code.
    It is not super important for understanding the basic fundamentals of deep learning.
    """
    
    nodes_postorder = []
    def recurse(node):
        if isinstance(node, Operation):
            for input_node in node.input_nodes:
                recurse(input_node)
        nodes_postorder.append(node)

    recurse(operation)
    return nodes_postorder

In [35]:
import numpy as np

In [26]:
class Session:
    
    def run(self, operation, feed_dict = {}):
        """ 
          operation: The operation to compute
          feed_dict: Dictionary mapping placeholders to input values (the data)  
        """
        
        # Puts nodes in correct order
        nodes_postorder = traverse_postorder(operation)
        
        for node in nodes_postorder:

            if type(node) == Placeholder:
                
                node.output = feed_dict[node]
                
            elif type(node) == Variable:
                
                node.output = node.value
                
            else: # Operation
                
                node.inputs = [input_node.output for input_node in node.input_nodes]

                 
                node.output = node.compute(*node.inputs)
                
            # Convert lists to numpy arrays
            if type(node.output) == list:
                node.output = np.array(node.output)
        
        # Return the requested node value
        return operation.output




In [28]:
sess = Session()

In [29]:
result = sess.run(operation=z,feed_dict={x:10})

In [30]:
result

101

In [31]:
g = Graph()

g.set_as_default()

A = Variable([[10,20],[30,40]])
b = Variable([1,1])

x = Placeholder()

y = matmul(A,x)

z = add(y,b)


In [32]:
sess = Session()

In [36]:
result = sess.run(operation=z,feed_dict={x:10})

In [37]:
result

array([[101, 201],
       [301, 401]])